In [225]:
import xml.etree.ElementTree as ElementTree

class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.updateShim( dict(parent_element.items()) )
        for element in parent_element:
            if len(element):
                aDict = XmlDictConfig(element)
            #   if element.items():
            #   aDict.updateShim(dict(element.items()))
                self.updateShim({element.tag: aDict})
            elif element.items():    # items() is specialy for attribtes
                elementattrib= element.items()
                if element.text:           
                    elementattrib.append((element.tag,element.text ))     # add tag:text if there exist
                self.updateShim({element.tag: dict(elementattrib)})
            else:
                self.updateShim({element.tag: element.text})

    def updateShim (self, aDict ):
        for key in aDict.keys():   # keys() includes tag and attributes
            if key in self:
                value = self.pop(key)
                if type(value) is not list:
                    listOfDicts = []
                    listOfDicts.append(value)
                    listOfDicts.append(aDict[key])
                    self.update({key: listOfDicts})
                else:
                    value.append(aDict[key])
                    self.update({key: value})
            else:
                self.update({key:aDict[key]})



In [227]:
import xml.etree.ElementTree as ElementTree

import xmltodict
import pprint
import json
import pandas as pd
import numpy as np



tree = ElementTree.parse('/Users/mina/Documents/NYU/BUILT/Matsim_Intermodal_comparison/raw/sample.xml')
root = tree.getroot()
xmldict = XmlDictConfig(root)

#xmldict['person']


In [228]:
# keep

id_data = []
activity_data = []
mode_data = []

for row in xmldict['person']:
    id_data.append(row['id'])
    activity_data.append(row['plan']['activity'])
    mode_data.append(row['plan']['leg'])


data_tuples = list(zip(id_data, activity_data, mode_data))

#-----------------------
# print(id_data[2])
# print(activity_data[2][1]['type'])
# print(len(mode_data[2]))
# print(mode_data[2])


In [229]:
## keep finish separating 
##need to change print to add
mode = ['walk','bike','car','taxi']
act = ['Home','Secondary','Work','University','School']

mode_mode_data = []
id_id_data=[]
activity_list = pd.DataFrame(columns=['type' , 'link', 'start_time' , 'end_time','activity'])



pt_count=0;
start = 0;
for i in range(0,len(activity_data)):
    start = 0;
    #print(id_data[i])
    
    for j in range(0,len(activity_data[i])):
        pt_count=0

        if (activity_data[i][j]['type'] in act):
            #print(activity_data[i][j])
            id_id_data.append(id_data[i])
            act_activity_data.append(activity_data[i][j]['type'])

            ###add activity to list
            new_list.append(activity_data[i][j])
            #print(new_list)
            
            ### append list to DataFrame
            activity_list= activity_list.append(new_list, ignore_index=True)
            new_list.clear()



            for k in range(start,len(mode_data[i])):

                if mode_data[i][k]['mode'] in mode:
                    mode_mode_data.append(mode_data[i][k]['mode'])
                    #print(i,j,k,mode_data[i][k])
                    start=k+1;
                    break;

                elif mode_data[i][k]['mode']=='pt': 
                    if(pt_count ==0):
                        mode_mode_data.append(mode_data[i][k]['mode'])
                        #print(i,j,k,mode_data[i][k])
                        pt_count=1 
                elif mode_data[i][k]['mode']=='egress_walk' and pt_count==1:
                    start = k+1;
                    break;

                else:
                    continue;


    mode_mode_data.append('')
    
    
print("data mining done")


data mining done


In [230]:
#id and mode data tuple
data_tuples = list(zip(id_id_data, mode_mode_data))

data_tuples

#keep -make a DataFrame

df = pd.DataFrame(data_tuples, columns=['id','mode'])



#df
print("dataframe succeed")

dataframe succeed


In [235]:
#merge index list and activity list

df_merged = pd.concat([df, activity_list], axis=1)
df_merged


,id,mode,type,link,start_time,end_time,activity
0,0,walk,Work,226193928_0,NaN,09:03:00,\n\t\t\t
1,0,walk,Work,420514148_1,15:34:22,18:44:22,\n\t\t\t
2,0,walk,Secondary,458166896_5,19:10:20,19:16:20,\n\t\t\t
3,0,,Work,588168464_0,20:05:14,NaN,\n\t\t\t
4,1,car,Home,5671084_0,NaN,06:03:00,\n\t\t\t
5,1,pt,Work,8948920_0,06:24:23,11:53:23,\n\t\t\t
6,1,walk,Work,5670719_4,14:25:05,16:38:05,\n\t\t\t
7,1,walk,Secondary,68668519_0,17:17:39,18:26:39,\n\t\t\t
8,1,,Work,630828584_1,19:04:56,NaN,\n\t\t\t
9,1007981,walk,Home,421854005_0,NaN,07:52:00,\n\t\t\t


In [237]:
df_merged['start_time']=pd.to_datetime(df_merged['start_time'])
df_merged['end_time']=pd.to_datetime(df_merged['end_time'])


#df_merged['end_time']=pd.to_datetime(df_merged['end_time'],format= '%H:%M:%S' ).dt.time





In [238]:
# check travel time
trave_time = pd.DataFrame(columns={'id','travel_time'})

for i in range(0,len(df_merged)-1):
    if(df_merged.iloc[i]['id']==df_merged.iloc[i+1]['id']):
        diff =pd.Timedelta(df_merged.iloc[i]['end_time']  - df_merged.iloc[i+1]['start_time'] ).seconds / 3600
    else:
        diff = np.nan
    
    trave_time=trave_time.append([{'id': df_merged.iloc[i]['id'], 'travel_time':diff}], ignore_index=True)




In [239]:
#merge 

df_final =  pd.concat([df_merged, trave_time['travel_time']], axis=1)
df_final

,id,mode,type,link,start_time,end_time,activity,travel_time
0,0,walk,Work,226193928_0,NaT,2019-03-18 09:03:00,\n\t\t\t,17.477222
1,0,walk,Work,420514148_1,2019-03-18 15:34:22,2019-03-18 18:44:22,\n\t\t\t,23.567222
2,0,walk,Secondary,458166896_5,2019-03-18 19:10:20,2019-03-18 19:16:20,\n\t\t\t,23.185000
3,0,,Work,588168464_0,2019-03-18 20:05:14,NaT,\n\t\t\t,NaN
4,1,car,Home,5671084_0,NaT,2019-03-18 06:03:00,\n\t\t\t,23.643611
5,1,pt,Work,8948920_0,2019-03-18 06:24:23,2019-03-18 11:53:23,\n\t\t\t,21.471667
6,1,walk,Work,5670719_4,2019-03-18 14:25:05,2019-03-18 16:38:05,\n\t\t\t,23.340556
7,1,walk,Secondary,68668519_0,2019-03-18 17:17:39,2019-03-18 18:26:39,\n\t\t\t,23.361944
8,1,,Work,630828584_1,2019-03-18 19:04:56,NaT,\n\t\t\t,NaN
9,1007981,walk,Home,421854005_0,NaT,2019-03-18 07:52:00,\n\t\t\t,23.622500


In [254]:
#group by id for travel_time

####### add mode on the final group by
total_travel_time = df_final.groupby(['id']).sum()
total_travel_time

#len(travel_time_comparison)

,travel_time
id,
0,64.229444
1,91.817778
1007981,46.448611
1007982,137.248056
1007983,66.963889
1007984,46.111667
1007985,137.789444
1007986,90.745556
1007987,90.149444


In [256]:
df_final.to_excel("sample_mode_final.xlsx")



In [257]:
total_travel_time.to_excel("sample_totalTime_final.xlsx")
#df_merged.to_csv("sample2.csv",sep='\t', encoding='utf-8')
print("all done")

all done
